In [1]:
import sys,os
import numpy as np
if(not sys.version_info[0]<3):
    from importlib import reload 
from bs4 import BeautifulSoup
import urllib
import requests
import pandas as pd
import nltk
import json

We will scrape the english haikus from the website being requested below.

In [2]:
page = requests.get('http://www.tempslibres.org/tl/tlphp/dbauteursl.php?lang=en&lg=e') 

In [3]:
soup=BeautifulSoup(page.content,'html.parser')

In [4]:
#print(soup.prettify())
#list(soup.children)

In [5]:
#id="content" and class='liensurl'

In [6]:
soup.find_all('td',class_='liensurl')
soup.find_all('tr')
print(len(soup.select('tr td.liensdesc'))) #[name, location, number of poems in database]
len(soup.select('tr td.liensurl'))#This gives the number of poets as it contains the url to their poems


1095


365

In [11]:
num_poets=len(soup.select('tr td.liensurl'))
poets=np.array([[soup.select('tr td.liensdesc')[i*3].get_text(),soup.select('tr td.liensdesc')[i*3+1].get_text(),'http://www.tempslibres.org/tl/tlphp/dbhk01.php?auteur='+soup.select('tr td.liensurl')[i].get_text()+'&lg=e'] for i in range(num_poets)])
#page is http://www.tempslibres.org/tl/tlphp/dbhk01.php?auteur=(****NAME****)&lg=e


In [12]:
poets[:3]

array([['Odd G. Aksnes ', 'Toensberg - Norway',
        'http://www.tempslibres.org/tl/tlphp/dbhk01.php?auteur=aksn-o &lg=e'],
       ['Melanie Alberts ', 'Austin, Texas - USA',
        'http://www.tempslibres.org/tl/tlphp/dbhk01.php?auteur=albe-m &lg=e'],
       ['Mohamad Alsari ', 'Alep - Syria',
        'http://www.tempslibres.org/tl/tlphp/dbhk01.php?auteur=alsa-m &lg=e']],
      dtype='<U67')

The following gets the poems from the website. It works ok when the poems are in english. Not otherwise, this is because it is a french website that offers traductions in the same html element. The traductions can something be given *before* the non-translated version. Kind of annoying.

The url has a lg=e at the end, i.e. I'm supposed to only have the poems that are in English, but their database ain't no bueno.

In [13]:
def findpoems(url):
    page=requests.get(url)
    poempage=BeautifulSoup(page.content,'html.parser').select('p.haiku')
    npoem=len(poempage)
    poems=[]
    for i in range(npoem):
        tmp=poempage[i].get_text()
        a=0
        for _ in range(3):
            a+=tmp[a:].find('\n')+2 #look for the third \n, this characterizes the end of a haiku.
        poems.append(tmp[:a].replace('\n',' \n '))
    return np.array(poems)

In [14]:
print(poets[2,1])
findpoems(poets[2,2])[:10] #Example of a non-english poet - From Syria

Alep - Syria


array(['Avec la vieille balançoire \n les souvenirs  \n vi',
       'ساعة بلا عقارب\r \n على نافذتك المهجورة \r \n ال',
       'horloge sans aiguilles \n sur ta fenêtre abandonnée \n la',
       'Maison abandonnée\r \n de la fenêtre cassée\r \n la',
       'بعد إطفاء الأضواء\r \n على واجهة متجر الأسلحة\r \n ال',
       "les lampes éteintes \n sur la vitrine d'armes \n la",
       "Deux ailes d'oiseau \n ton rouge à lèvres \n su",
       'خارجاً من البئر\r \n بالدلو أصطاد\r \n ال',
       "du puits \n dans un seau j'attrape \n une lune \n  \n ",
       'إشارة ربط الحزام \r \n كل ماتبقى \r \n عل'],
      dtype='<U73')

In [15]:
print(poets[7,1])
findpoems(poets[7,2])[:10] #poet from the USA.

Central Oregon - USA


array(['mountain cave -- \n from out of darkness \n the morning light \n  \n ',
       "after its first flight \n the young gerfalcon's talons \n tighter on my glove \n  \n "],
      dtype='<U81')

In [16]:
poets_USA=np.array([p for p in poets if 'USA' in p[1]])#I'll hope the authors of Murica all write in english.

In [20]:
import json
def createdatabase(poets,fp='peom_data.txt'):
    if(os.path.exists(fp)):
        os.remove(fp)
        f=open(fp,'w')
    else:
        f=open(fp,'w')
        
        #the above code deletes and recreate the file
        #we do this as the code belows just appends to it.
        #Without the above, running this function twice would add copies of the database.
        
    for i in range(len(poets)):
        poems=findpoems(poets[i,2])
        json_file={'Name': poets[i,0], 'Location': poets[i,1], 'Poems': poems.tolist()}
        json.dump(json_file,f,separators=(',',':'))
    f.close()
    return
def createdatabase_nponly(poets,fp='peom_data.txt'):
    poems=[]
    for i in range(len(poets)):
        poems=np.concatenate((poems,findpoems(poets[i,2])))
    np.save('poem_npdata',np.array(poems))
    return
%time createdatabase_nponly(poets_USA)

CPU times: user 7.26 s, sys: 256 ms, total: 7.51 s
Wall time: 2min 29s


Now we should have some sort of database of haikus. I uploaded the output of this function in Google Drive. It is under the name poem_data.txt .

In [31]:
poems_list=np.load('poem_npdata.npy')
npoem=len(poems_list)

In [52]:
dic=set(nltk.word_tokenize(poems_list[0]))
for i in range(npoem):
    dic= dic | set(nltk.word_tokenize(poems_list[i]))
len(dic)

5308

1) Either build an embedding from this or use other already embedded dictionary. I'll try an outside one.
2) Feed the Poems into an LSTM I guess